In [64]:
import numpy as np
from ReadBinary import *

def GetViewArrays(view):
    arrList = []
    meshList = []
    axisRange = [[], []]

    def AdjustAxisMinMax(axisRange, new_min_max):
        ax_min, ax_max = new_min_max
        if axisRange == []:
            axisRange.extend([ax_min, ax_max])
        else:
            if axisRange[0] > ax_min:
                axisRange[0] = ax_min
            if axisRange[1] < ax_max:
                axisRange[1] = ax_max

    if view["type"] == "partial":
        for gridName, params in gridParams.items():
            r0, r1 = params["r0"], params["r1"]
            plane = {"x":0, "y":1, "z":2}
            if r0[plane[view["plane"]]] <= view["at"] < r1[plane[view["plane"]]]:
                dataFileName =  "../data/3D/auto/" + \
                                gridName + "_" + view["arrayName"] + "_" + view["direction"] + \
                                "_@" + view["plane"] + "=" + str(view["at"]) + ".data"
                arr = GetArrays(dataFileName)
                H, V = None, None
                hRange, vRange = axisRange
                if view["plane"] == "x":
                    arr = arr[:, 0, :, :]
                    z = np.linspace(r0[2], r1[2], arr.shape[2])
                    y = np.linspace(r0[1], r1[1], arr.shape[1])
                    Z, Y = np.meshgrid(z, y, indexing="xy")
                    H, V = Z, Y
                    AdjustAxisMinMax(hRange, (r0[2], r1[2]))
                    AdjustAxisMinMax(vRange, (r0[1], r1[1]))
                        
                elif view["plane"] == "y":
                    arr = arr[:, :, 0, :]
                    z = np.linspace(r0[2], r1[2], arr.shape[2])
                    x = np.linspace(r0[0], r1[0], arr.shape[1])
                    Z, X = np.meshgrid(z, x, indexing="xy")
                    H, V = Z, X
                    AdjustAxisMinMax(hRange, (r0[2], r1[2]))
                    AdjustAxisMinMax(vRange, (r0[0], r1[0]))

                elif view["plane"] == "z":
                    arr = arr[:, :, :, 0]
                    y = np.linspace(r0[1], r1[1], arr.shape[2])
                    x = np.linspace(r0[0], r1[0], arr.shape[1])
                    X, Y = np.meshgrid(x, y, indexing="ij")
                    H, V = X, Y
                    AdjustAxisMinMax(hRange, (r0[0], r1[0]))
                    AdjustAxisMinMax(vRange, (r0[1], r1[1]))
                    
                else:
                    assert False
                arrList.append(arr)
                meshList.append([H, V])
                axisRange = [hRange, vRange]
                
    return arrList, meshList, axisRange


import pickle
fileParams = open("../data/3D/auto/params.param", "rb")
params = pickle.load(fileParams)
fileParams.close()

gridParams = params["grids"]
viewParams = params["views"]

print("views: ")
for i in range(len(viewParams)):
    print(i, " : ", viewParams[i])


views: 
0  :  {'type': 'partial', 'plane': 'x', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
1  :  {'type': 'partial', 'plane': 'y', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
2  :  {'type': 'partial', 'plane': 'z', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}


In [65]:
%pylab tk
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation

arrList, meshList, axisRange = GetViewArrays(viewParams[0])
plot_mag = False

arr_thresh = 0.0
def animate_arr(n):
    plt.clf()
    if plot_mag:
        arr_max = np.max([np.max(np.abs(arr[n])) for arr in arrList])
        if arr_thresh > 0 and arr_max > arr_thresh:
            arr_max = arr_thresh
        for ind in range(len(arrList)):
            H, V = meshList[ind]
            plt.pcolormesh(H, V, np.abs(arrList[ind][n]), cmap="rainbow", vmax=arr_max)
    else:
        arr_max = np.max([np.max(arr[n]) for arr in arrList])
        arr_min = np.min([np.min(arr[n]) for arr in arrList])
        if arr_thresh > 0 and np.abs(arr_max) > arr_thresh:
            arr_max = arr_thresh
        if arr_thresh > 0 and np.abs(arr_min) > arr_thresh:
            arr_min = -arr_thresh
        for ind in range(len(arrList)):
            H, V = meshList[ind]
            plt.pcolormesh(H, V, arrList[ind][n], cmap="rainbow", vmin=arr_min, vmax=arr_max)

    fig = plt.gcf()
    plt.colorbar()
    plt.pause(0.05)
    return fig

save_animation = True

Dh_m = axisRange[0][1] - axisRange[0][0] 
Dv_m = axisRange[1][1] - axisRange[1][0] 
if save_animation:
    fig = plt.figure(figsize=(7,7*(Dv_m/Dh_m)))
    anim = animation.FuncAnimation(fig, animate_arr, frames=arrList[0].shape[0], interval=1, repeat=False)
    anim.save("../data/3D/auto/" + 'Efield-anim.mp4', writer="ffmpeg", fps=15, dpi=200)
else:    
    plt.ion()
    plt.figure(figsize=(12,12*(Dv_m/Dh_m)))
    for n in range(arrList[0].shape[0]):
        animate_arr(n)


Populating the interactive namespace from numpy and matplotlib
